In [431]:
import pandas as pd
import datetime
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from geopy.distance import distance
import re

from ingeniring_utils import * # import all existing functions from file cleaning.py

%load_ext autoreload
%autoreload 2


pd.set_option('display.max_columns', 360)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


---



## Loading Info Data Set

In [432]:
info = pd.read_csv('../P3_data/Blight_Violations.csv',low_memory=False)

In [433]:
info.head(3)

,X,Y,ticket_id,ticket_number,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,violator_id,mailing_address_str_number,mailing_address_str_name,city,state,zip_code,non_us_str_code,country,violation_date,ticket_issued_time,hearing_date,hearing_time,judgment_date,violation_code,violation_description,disposition,clean_up_cost,payment_amount,payment_date,payment_status,collection_status,violation_address,parcelno,oid,geom,fine_amount
0,-83.072573,42.383350,18645,05001700DAH,"Buildings, Safety Engineering & Env Department",Orbie Gailes,Dean Byrd,601,KING,NaN,1614,599,King,Det,MI,48202,NaN,NaN,2005/01/13 00:00:00+00,1900/01/01 14:00:00,2005/02/07 00:00:00+00,09:00:00,2005/02/07 00:00:00+00,9-1-36(a),Failure of owner to obtain certificate of comp...,Responsible by Default,NaN,NaN,1970/01/01 00:00:00+00,NaN,NaN,601 KING,03002390.,18645,NaN,NaN
1,-83.072479,42.383394,18646,05001701DAH,"Buildings, Safety Engineering & Env Department",Orbie Gailes,Cynthia Roberts,607,KING,NaN,1615,18604,Appoline,Det,MI,48235,NaN,NaN,2005/01/14 00:00:00+00,1900/01/01 00:45:00,2005/02/14 00:00:00+00,09:00:00,2019/09/16 10:26:00+00,9-1-36(a),Failure of owner to obtain certificate of comp...,Not responsible by City Dismissal,NaN,NaN,1970/01/01 00:00:00+00,NO PAYMENT DUE,NaN,607 KING,03002391.,18646,NaN,NaN
2,-83.115061,42.359916,18648,05001703DAH,"Buildings, Safety Engineering & Env Department",Orbie Gailes,Dannny Barnes,4066,COLUMBUS,NaN,1617,4066,Columbus,Det,MI,48204,NaN,NaN,2005/01/14 00:00:00+00,1900/01/01 14:00:00,2005/02/14 00:00:00+00,09:00:00,2005/02/14 00:00:00+00,9-1-36(a),Failure of owner to obtain certificate of comp...,Not responsible by Dismissal,NaN,NaN,1970/01/01 00:00:00+00,NO PAYMENT DUE,NaN,4066 COLUMBUS,14002572.,18648,NaN,NaN


In [434]:
info.columns

Index(['X', 'Y', 'ticket_id', 'ticket_number', 'agency_name', 'inspector_name',
       'violator_name', 'violation_street_number', 'violation_street_name',
       'violation_zip_code', 'violator_id', 'mailing_address_str_number',
       'mailing_address_str_name', 'city', 'state', 'zip_code',
       'non_us_str_code', 'country', 'violation_date', 'ticket_issued_time',
       'hearing_date', 'hearing_time', 'judgment_date', 'violation_code',
       'violation_description', 'disposition', 'clean_up_cost',
       'payment_amount', 'payment_date', 'payment_status', 'collection_status',
       'violation_address', 'parcelno', 'oid', 'geom', 'fine_amount'],
      dtype='object')

In [435]:
columns_to_drop = ['ticket_number', 'agency_name', 'inspector_name',
       'violator_name', 'violation_street_number', 'violation_street_name',
       'violation_zip_code', 'violator_id', 'mailing_address_str_number',
       'mailing_address_str_name', 'city', 'state', 'zip_code',
       'non_us_str_code', 'country', 'violation_date', 'ticket_issued_time',
       'hearing_date', 'hearing_time', 'judgment_date', 'violation_code',
       'violation_description', 'disposition', 'clean_up_cost',
       'payment_amount', 'payment_date', 'payment_status', 'collection_status',
       'violation_address', 'oid', 'geom', 'fine_amount']

In [436]:
info = info.drop(columns = columns_to_drop)

In [437]:
info.rename({'Y':'lat', 'X':'lon', 'parcelno': 'parcel_id'}, axis=1, inplace=True)
info = info.dropna(subset=['parcel_id'])

In [438]:
info['parcel_id'] = info.apply(parse_parcel,axis=1)

In [439]:
info.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 306227 entries, 0 to 496334
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   lon        306227 non-null  float64
 1   lat        306227 non-null  float64
 2   ticket_id  306227 non-null  int64  
 3   parcel_id  306227 non-null  int64  
dtypes: float64(2), int64(2)
memory usage: 11.7 MB


In [440]:
info.head()

,lon,lat,ticket_id,parcel_id
0,-83.072573,42.383350,18645,3002390
1,-83.072479,42.383394,18646,3002391
2,-83.115061,42.359916,18648,14002572
3,-83.128041,42.393448,18649,12004708
6,-83.036142,42.374078,18652,13003490


-----





## Loading Blight Dataset

In [441]:
blight = pd.read_csv('data/Blight.csv', encoding='cp1252', low_memory=False)

In [442]:
blight.shape

(250306, 34)

In [443]:
blight.head(3)

,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,state,zip_code,non_us_str_code,country,ticket_issued_date,hearing_date,violation_code,violation_description,disposition,fine_amount,admin_fee,state_fee,late_fee,discount_amount,clean_up_cost,judgment_amount,payment_amount,balance_due,payment_date,payment_status,collection_status,grafitti_status,compliance_detail,compliance
0,22056,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","INVESTMENT INC., MIDWEST MORTGAGE",2900.0,TYLER,NaN,3.0,S. WICKER,CHICAGO,IL,60606,NaN,USA,2004-03-16 11:40:00,2005-03-21 10:30:00,9-1-36(a),Failure of owner to obtain certificate of comp...,Responsible by Default,250.0,20.0,10.0,25.0,0.0,0.0,305.0,0.0,305.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0
1,27586,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Michigan, Covenant House",4311.0,CENTRAL,NaN,2959.0,Martin Luther King,Detroit,MI,48208,NaN,USA,2004-04-23 12:30:00,2005-05-06 13:30:00,61-63.0600,Failed To Secure Permit For Lawful Use Of Buil...,Responsible by Determination,750.0,20.0,10.0,75.0,0.0,0.0,855.0,780.0,75.0,2005-06-02 00:00:00,PAID IN FULL,NaN,NaN,compliant by late payment within 1 month,1.0
2,22062,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","SANDERS, DERRON",1449.0,LONGFELLOW,NaN,23658.0,P.O. BOX,DETROIT,MI,48223,NaN,USA,2004-04-26 13:40:00,2005-03-29 10:30:00,9-1-36(a),Failure of owner to obtain certificate of comp...,Not responsible by Dismissal,250.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN


In [444]:
blight.columns

Index(['ticket_id', 'agency_name', 'inspector_name', 'violator_name',
       'violation_street_number', 'violation_street_name',
       'violation_zip_code', 'mailing_address_str_number',
       'mailing_address_str_name', 'city', 'state', 'zip_code',
       'non_us_str_code', 'country', 'ticket_issued_date', 'hearing_date',
       'violation_code', 'violation_description', 'disposition', 'fine_amount',
       'admin_fee', 'state_fee', 'late_fee', 'discount_amount',
       'clean_up_cost', 'judgment_amount', 'payment_amount', 'balance_due',
       'payment_date', 'payment_status', 'collection_status',
       'grafitti_status', 'compliance_detail', 'compliance'],
      dtype='object')

In [445]:
list_to_remove_blight = ['inspector_name', 'violator_name',
       'violation_street_number', 'violation_street_name',
       'violation_zip_code', 'mailing_address_str_number',
       'mailing_address_str_name', 'city', 'state', 'zip_code',
       'non_us_str_code', 'country', 'ticket_issued_date', 'hearing_date',
       'violation_code', 'violation_description', 'payment_amount', 'clean_up_cost',
       'payment_status', 'late_fee', 'fine_amount', 'admin_fee',
        'state_fee',              
       'grafitti_status', 'collection_status', 'payment_date', 'payment_status', 'compliance_detail', 'balance_due']

In [446]:
blight = blight.drop(columns=list_to_remove_blight, axis=1)

In [447]:
blight = blight.dropna(subset=['compliance'])

In [448]:
blight.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159880 entries, 0 to 250293
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   ticket_id        159880 non-null  int64  
 1   agency_name      159880 non-null  object 
 2   disposition      159880 non-null  object 
 3   discount_amount  159880 non-null  float64
 4   judgment_amount  159880 non-null  float64
 5   compliance       159880 non-null  float64
dtypes: float64(3), int64(1), object(2)
memory usage: 8.5+ MB


In [449]:
blight_info = pd.merge(blight, info, left_on='ticket_id', right_on='ticket_id', how= 'left')

In [450]:
blight_info.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159880 entries, 0 to 159879
Data columns (total 9 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   ticket_id        159880 non-null  int64  
 1   agency_name      159880 non-null  object 
 2   disposition      159880 non-null  object 
 3   discount_amount  159880 non-null  float64
 4   judgment_amount  159880 non-null  float64
 5   compliance       159880 non-null  float64
 6   lon              97398 non-null   float64
 7   lat              97398 non-null   float64
 8   parcel_id        97398 non-null   float64
dtypes: float64(6), int64(1), object(2)
memory usage: 12.2+ MB


In [451]:
blight_info =  blight_info.dropna(subset=['parcel_id'])

In [452]:
blight_info.parcel_id = blight_info.parcel_id.astype(int, copy = False )

-----





## Exploring Motor City Mapping

In [453]:
mcm = pd.read_csv('../P3_data/Motor_City_Mapping,_Winter_2013-14_Certified_Results.csv', low_memory=False)

In [454]:
mcm.head(3)

,OBJECTID,D3_SurveyParcelID,CityParcelID2013,AddCombo,Address_Number,Address_Direction,Street,Zip,PCount,Blext_ID,User_ID,Reviewer_ID,Time_Surveyed,Updated_At,Status,Photo_URL,Structure,Use_,Condition,Occupancy,Needs_Boarding,Fire,Dumping,Units,Improved,Maintained,PublicUse,Notes,GEOID10_Tract,NAMELSAD10_Tract,GEOID10_Block,NAME10_Block,GeoID10_BG
0,1,01000004.001,01000004.001,201 RANDOLPH,201,,RANDOLPH,48200,1,411094.0,2891.0,3284.0,2014/02/07 17:58:18+00,2014/08/27 15:31:15+00,1.0,https://wdwot.s3.amazonaws.com/blexts/000/411/...,yes,commercial,good,occupied,no,no,no,NaN,NaN,NaN,NaN,Parcel contains portion of Detroit-Windsor Tunnel,26163517200,Census Tract 5172,261635172001080,Block 1080,261635172001
1,2,01000004.002L,01000004.002L,159 RANDOLPH,159,,RANDOLPH,48200,1,411082.0,2891.0,3284.0,2014/02/07 17:57:31+00,2014/02/10 18:16:20+00,1.0,https://wdwot.s3.amazonaws.com/blexts/000/411/...,yes,commercial,good,occupied,no,no,no,NaN,NaN,NaN,NaN,NaN,26163517200,Census Tract 5172,261635172001082,Block 1082,261635172001
2,3,01000005.002L,01000005.002L,160 E JEFFERSON,160,E,JEFFERSON,,1,411127.0,2891.0,3284.0,2014/02/07 18:01:08+00,2014/02/10 18:15:49+00,1.0,https://wdwot.s3.amazonaws.com/blexts/000/411/...,yes,commercial,good,occupied,no,no,no,NaN,NaN,NaN,NaN,NaN,26163517200,Census Tract 5172,261635172001088,Block 1088,261635172001


In [455]:
# Selecting not important features to drop from DF
columns_to_keep = [
                     'CityParcelID2013',
                     'AddCombo',
                     'Structure',
                     'Use_',
                     'Condition',
                     'Occupancy',
                     'Needs_Boarding',
                     'Fire',
                     'Dumping',
                     'Units',
                     'Improved',
                     'Maintained',
                    ]

In [456]:
mcm = mcm[columns_to_keep]
mcm.rename({'CityParcelID2013': 'parcel_id'}, axis=1, inplace=True)
mcm['parcel_id'] = mcm.apply(parse_parcel,axis=1)
mcm = mcm.dropna(subset= [ 'Condition', 'Occupancy', 'Needs_Boarding', 'Fire', 'Dumping' ], how='all')

In [457]:
mcm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 373177 entries, 0 to 379548
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   parcel_id       373177 non-null  int64 
 1   AddCombo        373177 non-null  object
 2   Structure       373175 non-null  object
 3   Use_            373171 non-null  object
 4   Condition       261355 non-null  object
 5   Occupancy       261202 non-null  object
 6   Needs_Boarding  261271 non-null  object
 7   Fire            261284 non-null  object
 8   Dumping         373169 non-null  object
 9   Units           245275 non-null  object
 10  Improved        112019 non-null  object
 11  Maintained      111970 non-null  object
dtypes: int64(1), object(11)
memory usage: 37.0+ MB


In [458]:
count_id = pd.DataFrame(mcm.groupby('parcel_id')['parcel_id'].count())
count_id = count_id.rename({'parcel_id':'parcel_count'}, axis=1).reset_index()

mcm = pd.merge(mcm, count_id, left_on='parcel_id', right_on='parcel_id')
mcm = mcm[mcm.parcel_count == 1].drop('parcel_count', axis =1)

In [459]:
mcm.Improved.fillna('no', inplace = True)
mcm.Maintained.fillna('no', inplace = True)

mcm.Units.fillna('4+ units', inplace = True)
mcm.Fire.fillna('no', inplace = True)

mcm.dropna(subset=['Condition'], inplace=True)
mcm.Needs_Boarding.fillna('no', inplace=True)

In [460]:
mcm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 247393 entries, 2 to 373176
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   parcel_id       247393 non-null  int64 
 1   AddCombo        247393 non-null  object
 2   Structure       247392 non-null  object
 3   Use_            247392 non-null  object
 4   Condition       247393 non-null  object
 5   Occupancy       247217 non-null  object
 6   Needs_Boarding  247393 non-null  object
 7   Fire            247393 non-null  object
 8   Dumping         247386 non-null  object
 9   Units           247393 non-null  object
 10  Improved        247393 non-null  object
 11  Maintained      247393 non-null  object
dtypes: int64(1), object(11)
memory usage: 24.5+ MB


In [461]:
mcm.head()

,parcel_id,AddCombo,Structure,Use_,Condition,Occupancy,Needs_Boarding,Fire,Dumping,Units,Improved,Maintained
2,1000005,160 E JEFFERSON,yes,commercial,good,occupied,no,no,no,4+ units,no,no
3,1000007,110 E JEFFERSON,yes,commercial,good,occupied,no,no,no,4+ units,no,no
49,1004178,3100 WOODWARD AVENUE 14,yes,commercial,good,occupied,no,no,no,4+ units,no,no
60,1004189,3670 WOODWARD AVENUE 28/3,yes,commercial,good,occupied,no,no,no,4+ units,no,no
111,1001799,285 PIQUETTE,yes,mixed,good,occupied,no,no,no,4+ units,no,no


In [462]:
df = pd.merge(blight_info, mcm, left_on='parcel_id', right_on='parcel_id', how='left')

In [463]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 97398 entries, 0 to 97397
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ticket_id        97398 non-null  int64  
 1   agency_name      97398 non-null  object 
 2   disposition      97398 non-null  object 
 3   discount_amount  97398 non-null  float64
 4   judgment_amount  97398 non-null  float64
 5   compliance       97398 non-null  float64
 6   lon              97398 non-null  float64
 7   lat              97398 non-null  float64
 8   parcel_id        97398 non-null  int64  
 9   AddCombo         78863 non-null  object 
 10  Structure        78863 non-null  object 
 11  Use_             78862 non-null  object 
 12  Condition        78863 non-null  object 
 13  Occupancy        78823 non-null  object 
 14  Needs_Boarding   78863 non-null  object 
 15  Fire             78863 non-null  object 
 16  Dumping          78859 non-null  object 
 17  Units       

In [464]:
df = df.dropna(subset= ['Use_', 'Maintained', 'Improved', 'Units', 'Dumping', 'Fire', 'Needs_Boarding', 'Occupancy', 'Condition', 'AddCombo', 'Structure'])

In [465]:
# df.to_csv(r'data/df.csv', index = False)

In [466]:
df.shape

(78818, 20)

-----





## Exploring Reported_Major_Crimes_2011_to_2014

In [467]:
crimes = pd.read_csv('../P3_data/Reported_Major_Crimes_2011_to_2014.csv', low_memory=False)

In [468]:
crimes.head()

,ROWNUM,CATEGORY,OFFENSEDESCRIPTION,STATEOFFENSEFILECLASS,INCIDENTDATE,HOUR,SCA,PRECINCT,NEIGHBORHOOD,CENSUSTRACT,LOCATION,ObjectId
0,1311873,STOLEN VEHICLE,VEHICLE THEFT,24001,2011/01/02 00:00:00+00,0,1103.0,11.0,PERSHING,5066.0,"00 EUREKA/E LANTZ\n(42.4382, -83.0579)",1
1,1311874,AGGRAVATED ASSAULT,AGG/FEL ASSAULT - NON-FAMILY - OTHER WEAPON,13002,2011/01/02 00:00:00+00,11,709.0,7.0,LAFAYETTE PARK,5166.0,"E.LAFAYETTE01800\n(42.3394, -83.0295)",2
2,1311878,LARCENY,"LARCENY - FROM BUILDING (INCLUDES LIBRARY, OFF...",23003,2011/01/02 00:00:00+00,11,1105.0,11.0,OUTER DRIVE VAN DYKE,5051.0,"VANDYKE19900\n(42.4415, -83.0238)",3
3,1311879,LARCENY,LARCENY (OTHER),23007,2011/01/01 00:00:00+00,20,706.0,7.0,CHENE,5188.0,"JOS CAMPAU04100\n(42.3606, -83.0339)",4
4,1311883,STOLEN VEHICLE,VEHICLE THEFT,24001,2011/01/02 00:00:00+00,3,612.0,6.0,WARRENDALE,5457.0,"PIEDMONT06000\n(42.3328, -83.228)",5


In [469]:
crimes['lon'] = crimes.LOCATION.map(extract_long_crime)
crimes['lat'] = crimes.LOCATION.map(extract_lat_crime)

In [470]:
crimes.head(3)

,ROWNUM,CATEGORY,OFFENSEDESCRIPTION,STATEOFFENSEFILECLASS,INCIDENTDATE,HOUR,SCA,PRECINCT,NEIGHBORHOOD,CENSUSTRACT,LOCATION,ObjectId,lon,lat
0,1311873,STOLEN VEHICLE,VEHICLE THEFT,24001,2011/01/02 00:00:00+00,0,1103.0,11.0,PERSHING,5066.0,"00 EUREKA/E LANTZ\n(42.4382, -83.0579)",1,-83.0579,42.4382
1,1311874,AGGRAVATED ASSAULT,AGG/FEL ASSAULT - NON-FAMILY - OTHER WEAPON,13002,2011/01/02 00:00:00+00,11,709.0,7.0,LAFAYETTE PARK,5166.0,"E.LAFAYETTE01800\n(42.3394, -83.0295)",2,-83.0295,42.3394
2,1311878,LARCENY,"LARCENY - FROM BUILDING (INCLUDES LIBRARY, OFF...",23003,2011/01/02 00:00:00+00,11,1105.0,11.0,OUTER DRIVE VAN DYKE,5051.0,"VANDYKE19900\n(42.4415, -83.0238)",3,-83.0238,42.4415


In [471]:
crimes[['lat','lon']].describe()

,lat,lon
count,234978.000000,234978.000000
mean,1833.966961,1708.699648
std,42288.290069,42293.597149
min,39.744700,-84.912300
25%,42.356000,-83.188300
50%,42.391800,-83.111700
75%,42.421300,-83.027100
max,999999.000100,999999.000100


In [473]:
crimes =  crimes[ (crimes['lat'] < 45) | (crimes['lon'] < 85) ]
crimes['coordinates'] = crimes.apply(crime_coord, axis=1)

In [474]:
df['coordinates'] = df.apply(crime_coord, axis=1)

In [472]:
# crimes.INCIDENTDATE = pd.to_datetime(crimes['INCIDENTDATE']).dt.date
# crimes.INCIDENTDATE = pd.to_datetime(crimes.INCIDENTDATE)

In [476]:
crimes = crimes[['lat','lon', 'NEIGHBORHOOD']]

In [477]:
neigh_crimes = crimes.groupby('NEIGHBORHOOD').count()

In [478]:
neigh_crimes['lat'] = crimes.groupby('NEIGHBORHOOD')['lat'].mean()
neigh_crimes['lon'] = crimes.groupby('NEIGHBORHOOD')['lon'].mean()

In [479]:
neigh_crimes['coordinates'] = neigh_crimes.apply(crime_coord, axis=1)

In [481]:
neigh_crimes.head()

,lat,lon,coordinates
NEIGHBORHOOD,,,
ARDEN PARK/EAST BOSTON,42.384906,-83.078589,"(42.384905825242676, -83.07858859223302)"
ATKINSON-EUCLID,42.378886,-83.083864,"(42.3788859688196, -83.08386391982192)"
BAGLEY,42.423409,-83.151066,"(42.42340907749086, -83.15106608856087)"
BALDUCK PARK,42.414952,-82.920932,"(42.414951831750244, -82.92093202170992)"
BARTON MCFARLANE,42.362512,-83.163713,"(42.36251198038304, -83.16371272769739)"


In [484]:
df['crime_count'] = df.apply(crime_counts, args= [neigh_crimes.coordinates.values], axis=1)

In [485]:
df.head()

,ticket_id,agency_name,disposition,discount_amount,judgment_amount,compliance,lon,lat,parcel_id,AddCombo,Structure,Use_,Condition,Occupancy,Needs_Boarding,Fire,Dumping,Units,Improved,Maintained,coordinates,crime_count
0,22056,"Buildings, Safety Engineering & Env Department",Responsible by Default,0.0,305.0,0.0,-83.124350,42.390788,12004359,2900 TYLER,yes,residential,poor,unoccupied,yes,yes,no,4+ units,no,no,"(42.390787911000096, -83.124350291)",4
1,27586,"Buildings, Safety Engineering & Env Department",Responsible by Determination,0.0,855.0,1.0,-83.135099,42.326928,20004897,4311 CENTRAL,yes,institutional,good,occupied,no,no,no,4+ units,no,no,"(42.3269276180001, -83.13509917499991)",1
4,18735,"Buildings, Safety Engineering & Env Department",Responsible by Default,0.0,140.0,0.0,-83.037736,42.388692,15014168,8228 MT ELLIOTT,yes,commercial,good,occupied,no,no,no,4+ units,no,no,"(42.388692329, -83.037735508)",0
5,18733,"Buildings, Safety Engineering & Env Department",Responsible by Default,0.0,140.0,0.0,-83.037736,42.388692,15014168,8228 MT ELLIOTT,yes,commercial,good,occupied,no,no,no,4+ units,no,no,"(42.388692329, -83.037735508)",0
7,18741,"Buildings, Safety Engineering & Env Department",Responsible by Default,0.0,855.0,0.0,-83.180489,42.440187,22030316,20024 SCHAEFER,yes,residential,good,occupied,no,no,no,1 unit,no,no,"(42.4401871680001, -83.18048924)",1


In [487]:
df.crime_count.unique()

array([4, 1, 0, 3, 2, 9, 8, 7, 5, 6])

In [488]:
df.to_csv(r'data/df.csv', index = False)